####  O que é Pandera?
- Pandera é uma biblioteca Python open-source projetada para validação de dados em DataFrames do pandas. Ela permite definir esquemas (schemas) para verificar tipos de dados, valores nulos, restrições personalizadas e testes estatísticos, ajudando a garantir a integridade dos dados em pipelines de ETL, machine learning e análise de dados. 
- É inspirada em bibliotecas como Great Expectations e Pydantic, e integra-se perfeitamente com o pandas.

In [1]:
import pandera.pandas as pa  # <- novo
# opcional: você pode usar os checks via pa.Check


In [2]:
import os
os.environ["DISABLE_PANDERA_IMPORT_WARNING"] = "True"
import pandera.pandas as pa


In [3]:
#%pip install -U pip setuptools wheel
#%pip install numpy pandas pandera


In [4]:
import pandera.pandas as pa
from pandera import Check  # ainda funciona; alternativamente use pa.Check


In [5]:
import pandas as pd, pandera as pa
print(pd.__version__, pa.__version__)


2.3.3 0.26.1


In [6]:
import pandas as pd
import pandera.pandas as pa  # novo import

schema = pa.DataFrameSchema(
    {
        "idUsuario": pa.Column(
            pa.Int64,
            checks=[pa.Check.ge(1)],
            nullable=False,
            coerce=True,
            unique=True,
        ),
        "nomeEvento": pa.Column(
            str,
            checks=[pa.Check.str_length(min_value=1, max_value=200)],
            nullable=False,
            coerce=True,
        ),
        "dataHora": pa.Column(
            pa.DateTime,
            nullable=False,
            coerce=True,
        ),
        "metadado": pa.Column(
            object,
            nullable=True,
            checks=pa.Check(
                lambda x: pd.isna(x) or isinstance(x, dict),
                element_wise=True,
                error="metadado deve ser dict (ou NA).",
            ),
        ),
    },
    strict=True,
    ordered=False,
)


### Observações:
- coerce=True padroniza tipos na entrada (excelente para dados CSV/JSON).

- Para regex em dataHora, sua ideia funciona, mas usar pa.DateTime é mais seguro e cobre timezone, milissegundos etc.

In [7]:
import pandera as pa

df = pd.DataFrame({
    "idUsuario": [1, 2, 3],
    "nomeEvento": ["login", "logout", "pageview"],
    "dataHora": ["2025-10-31T09:47:00", "2025-10-31T10:00:00Z", "2025-10-31 10:15:00-03:00"],
    "metadado": [ {"ip":"1.2.3.4"}, None, {"browser":"Firefox"} ],
})

try:
    df_valid = schema.validate(df, lazy=True)  # lazy coleta todos os erros
    print("OK! Dados válidos.")
except pa.errors.SchemaErrors as err:
    print("Erros de validação:")
    print(err.failure_cases)  # dataframe com detalhes

Erros de validação:
     column failure_case index schema_context                           check  \
0  dataHora         None  None         Column  coerce_dtype('datetime64[ns]')   
1  dataHora       object  None         Column         dtype('datetime64[ns]')   

  check_number  
0         None  
1         None  


In [8]:
import pandera as pa

schema_1 = pa.DataFrameSchema({
    "nomeEvento": pa.Column(
        str,
        checks=[
            pa.Check.isin({"login","logout","pageview"}),          # enum
            pa.Check.str_length(min_value=1, max_value=200),       # tamanho
            pa.Check.str_matches(r"^[a-z]+(?:_[a-z]+)*$"),         # snake_case
        ],
        coerce=True,
    )
})


c:\Users\luciana.santos\AppData\Local\anaconda3\Lib\site-packages\pandera\_pandas_deprecated.py:149: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [9]:
display(schema_1)

<Schema DataFrameSchema(columns={'nomeEvento': <Schema Column(name=nomeEvento, type=DataType(str))>}, checks=[], parsers=[], index=None, dtype=None, coerce=False, strict=False, name=None, ordered=False, unique=None, report_duplicates=all, unique_column_names=False, add_missing_columns=False, title=None, description=None, metadata=None, drop_invalid_rows=False)>

In [10]:
print(schema_1)

<Schema DataFrameSchema(
    columns={
        'nomeEvento': <Schema Column(name=nomeEvento, type=DataType(str))>
    },
    checks=[],
    parsers=[],
    coerce=False,
    dtype=None,
    index=None,
    strict=False,
    name=None,
    ordered=False,
    unique_column_names=False,
    metadata=None, 
    add_missing_columns=False
)>


In [11]:
schema = pa.DataFrameSchema({
    "idUsuario": pa.Column(pa.Int64, checks=[pa.Check.ge(1)], unique=True, coerce=True),
    "tempoSessaoSeg": pa.Column(pa.Int64, checks=[pa.Check.ge(0)], coerce=True),
    "taxaConversao": pa.Column(float, checks=[pa.Check.between(0, 1)], coerce=True),
})


In [12]:
print(schema)

<Schema DataFrameSchema(
    columns={
        'idUsuario': <Schema Column(name=idUsuario, type=DataType(int64))>
        'tempoSessaoSeg': <Schema Column(name=tempoSessaoSeg, type=DataType(int64))>
        'taxaConversao': <Schema Column(name=taxaConversao, type=DataType(float64))>
    },
    checks=[],
    parsers=[],
    coerce=False,
    dtype=None,
    index=None,
    strict=False,
    name=None,
    ordered=False,
    unique_column_names=False,
    metadata=None, 
    add_missing_columns=False
)>


In [13]:
try:
    df_ok = schema.validate(df, lazy=True)
except pa.errors.SchemaErrors as e:
    print("Falhas:\n", e.failure_cases)  # DataFrame com detalhes


Falhas:
   column    failure_case index   schema_context                check  \
0   None  tempoSessaoSeg  None  DataFrameSchema  column_in_dataframe   
1   None   taxaConversao  None  DataFrameSchema  column_in_dataframe   

  check_number  
0         None  
1         None  


In [15]:
schema_eventos = pa.DataFrameSchema(
    {
        "idUsuario": pa.Column(pa.Int64, checks=[pa.Check.ge(1)], coerce=True),
        "nomeEvento": pa.Column(
            str,
            checks=[pa.Check.isin({"login","logout","pageview"}),
                    pa.Check.str_length(1, 200)],
            coerce=True,
        ),
        "dataHora": pa.Column(pa.DateTime, coerce=True),
        "metadado": pa.Column(object, nullable=True),
    },
    checks=[
        pa.Check(
            lambda df: not df.duplicated(subset=["idUsuario","dataHora"]).any(),
            error="(idUsuario, dataHora) deve ser único."
        ),
        pa.Check(
            lambda df: (
                (~(df["nomeEvento"] == "login")) |
                (df["metadado"].apply(lambda x: isinstance(x, dict) and "ip" in x))
            ).all(),
            error="Eventos 'login' devem ter metadado com 'ip'."
        ),
    ],
    strict=True,
)


In [16]:
print(schema_eventos)

<Schema DataFrameSchema(
    columns={
        'idUsuario': <Schema Column(name=idUsuario, type=DataType(int64))>
        'nomeEvento': <Schema Column(name=nomeEvento, type=DataType(str))>
        'dataHora': <Schema Column(name=dataHora, type=DataType(datetime64[ns]))>
        'metadado': <Schema Column(name=metadado, type=DataType(object))>
    },
    checks=[
        <Check <lambda>: (idUsuario, dataHora) deve ser único.>
        <Check <lambda>: Eventos 'login' devem ter metadado com 'ip'.>
    ],
    parsers=[],
    coerce=False,
    dtype=None,
    index=None,
    strict=True,
    name=None,
    ordered=False,
    unique_column_names=False,
    metadata=None, 
    add_missing_columns=False
)>
